<a href="https://colab.research.google.com/github/joy-deep-cs/IITK-CS-779/blob/tanmay/dataCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from lxml import etree
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from collections import Counter
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

cloning git repository

In [1]:
!git clone https://github.com/harperco/MeasEval.git

Cloning into 'MeasEval'...
remote: Enumerating objects: 2245, done.
remote: Counting objects: 100% (2245/2245), done.
remote: Compressing objects: 100% (1962/1962), done.
remote: Total 2245 (delta 211), reused 2244 (delta 210), pack-reused 0
Receiving objects: 100% (2245/2245), 659.60 KiB | 5.74 MiB/s, done.
Resolving deltas: 100% (211/211), done.


In [37]:
import glob, os    
#df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "/content/MeasEval/data/train/tsv/*.tsv"))))
dfAnnotation = pd.concat(map(lambda file: pd.read_csv(file,delimiter='\t',engine='python'), glob.glob(os.path.join('', "/content/MeasEval/data/train/tsv/*.tsv"))))

,docId,annotSet,annotType,startOffset,endOffset,annotId,text,other
0,S0378112713005288-1720,1,Quantity,334,336,T1-1,15,"{""mods"": [""IsCount""]}"
1,S0378112713005288-1720,1,MeasuredEntity,337,349,T2-1,upland sites,"{""HasQuantity"": ""T1-1""}"
2,S0378112713005288-1720,2,Quantity,374,388,T1-2,120 m to 380 m,"{""mods"": [""IsRange""], ""unit"": ""m""}"
3,S0378112713005288-1720,2,MeasuredProperty,351,360,T3-2,altitudes,"{""HasQuantity"": ""T1-2""}"
4,S0378112713005288-1720,2,MeasuredEntity,337,349,T2-2,upland sites,"{""HasProperty"": ""T3-2""}"


In [38]:
import glob
import pandas as pd
data = []
path = '/content/MeasEval/data/train/text/'
files = [f for f in os.listdir(path)]
for f in files:
  with open (path+f, "r") as myfile:
    data.append([f.strip('.txt'),myfile.read()])

In [40]:
dfSentence = pd.DataFrame(data, columns =['docId', 'Text']) 

In [43]:
dfAnnotation

,docId,annotSet,annotType,startOffset,endOffset,annotId,text,other
0,S0378112713005288-1720,1,Quantity,334,336,T1-1,15,"{""mods"": [""IsCount""]}"
1,S0378112713005288-1720,1,MeasuredEntity,337,349,T2-1,upland sites,"{""HasQuantity"": ""T1-1""}"
2,S0378112713005288-1720,2,Quantity,374,388,T1-2,120 m to 380 m,"{""mods"": [""IsRange""], ""unit"": ""m""}"
3,S0378112713005288-1720,2,MeasuredProperty,351,360,T3-2,altitudes,"{""HasQuantity"": ""T1-2""}"
4,S0378112713005288-1720,2,MeasuredEntity,337,349,T2-2,upland sites,"{""HasProperty"": ""T3-2""}"
...,...,...,...,...,...,...,...,...
7,S0025322712001600-2406,3,MeasuredProperty,810,816,T3-3,weight,"{""HasQuantity"": ""T1-3""}"
8,S0025322712001600-2406,3,MeasuredEntity,767,778,T4-3,uncertainty,"{""HasProperty"": ""T3-3""}"
9,S0025322712001600-2406,4,Quantity,824,827,T1-4,95%,"{""unit"": ""%""}"
10,S0025322712001600-2406,4,MeasuredProperty,828,844,T3-4,confidence level,"{""HasQuantity"": ""T1-4""}"


In [44]:
dfSentence

,docId,Text
0,S0012821X12004384-1148,Environmental changes associated with the Pale...
1,S0927775713009606-1216,Wide-angle diffraction patterns were acquired ...
2,S0167610512002292-3305,The results from three different simulations a...
3,S2211124712002884-649,(F) Intracellular PGRN levels in iPSC-derived ...
4,S1389128612002496-5994,The results in Table 3 indicate that when ther...
...,...,...
223,S0016236113008041-3012,The fate of trace elements was investigated in...
224,S0167819113001051-1247,"Although each node has 18 cores, the intended ..."
225,S0006322312001096-1248,"After adjustment for multiple covariates (age,..."
226,S0025322712001600-2406,Analysis of the diffraction data was conducted...


In [47]:
sen_anno_map = {}
def get_sen_anno(x):
    sen_id = x['docId']
    ent_type = x['annotType']
    startOffset = x['startOffset']
    endOffset = x['endOffset']
    if sen_id not in sen_anno_map:
        sen_anno_map[sen_id] = [[],[]]
        
    ent_text = x['text']

    if ent_type=='Quantity':
        sen_anno_map[sen_id][1].extend([(startOffset,endOffset,ent_text)])
    elif ent_type =='MeasuredEntity':
        sen_anno_map[sen_id][0].extend([(startOffset,endOffset,ent_text)])
        
    

t =dfAnnotation.apply(get_sen_anno,axis=1)




In [51]:
test_data = []
def map_to_sen(x):
    
    sen_text = x['Text']
    
    sen_id = x['docId']
    
    sen_ent = [[],[]]
    
    if sen_id in sen_anno_map:
        sen_ent = sen_anno_map[sen_id]
    
    MeasuredEntity_list = sen_ent[0]
    Quantity_list = sen_ent[1]
    
    #MeasuredEntity_list = [(int(a[0])-sen_off+1, int(a[1])-sen_off+1,a[2]) for a in MeasuredEntity_list]
    #Quantity_list = [( int(a[0])-sen_off+1, int(a[1])-sen_off+1,a[2]) for a in Quantity_list]
    
    test_data.append((sen_text,  MeasuredEntity_list, Quantity_list, sen_id))

t =dfSentence.apply(map_to_sen ,axis=1)

In [62]:
def get_token_tags(sen):
    sen1 = []
    tags = []
    text = sen[0]
    tagged_entities1 = sen[1]
    
    tagged_entities = sen[2]
    
    text = re.sub('/|-', ' ', text)
    
    sen_tokens = word_tokenize(text)
    
    offset = 0
    for token in sen_tokens:
        label = 'O'
        
        try:
            ind = text[offset:].index(token)
        except:
            if len(token)>2:
                print("token not present in string",token)
                
#                 print(text)
#                 print(text[offset:])
                
            ind = 0
        
        start = offset+ind
        end = start+len(token)
        
        #print(token,start,end)
        
        i = 0
        for i , adr in enumerate(tagged_entities):
            if adr[0]>start:
                continue
            elif adr[1]<=start:
                continue
            else:
                label = "MeasuredEntity"

                
        i = 0
        for i , adr in enumerate(tagged_entities1):
            if adr[0]>start:
                continue
            elif adr[1]<=start:
                continue
            else:
                label = "Quantity"
                
        offset = end-1
        
        if len(token)>0:
            sen1.append(token)
            tags.append(label)
            
    return sen1, tags

In [63]:
def get_doc_wise_tokenized_data(test_data):
    tokenised_test_data = []
    for sen_no, sen in enumerate(test_data):

        sen1, tags = get_token_tags(sen)

        tokenised_test_data.append((sen,sen1,tags))

    document_data = {}

    for row in tokenised_test_data:

        document_id = row[0][3][1]

        if document_id not in document_data:
            document_data[document_id] = []
        document_data[document_id].append(row)
        
    return document_data

In [64]:
document_data_lexero = get_doc_wise_tokenized_data(test_data)

dict_items([('0', [(('Environmental changes associated with the Paleocene–Eocene thermal maximum (PETM, ∼56 Ma) have not yet been documented in detail from the North Sea Basin. Located within proximity to the North Atlantic igneous province (NAIP), the Kilda Basin, and the northern rain belt (paleolatitude 54 °N) during the PETM, this is a critical region for testing proposed triggers of atmospheric carbon release that may have caused the global negative carbon isotope excursion (CIE) in marine and terrestrial environments. The CIE onset is identified from organic matter δ13C in exceptional detail within a highly expanded sedimentary sequence. Pollen and spore assemblages analysed in the same samples for the first time allow a reconstruction of possible changes to vegetation on the surrounding landmass. Multiproxy palynological, geochemical, and sedimentologic records demonstrate enhanced halocline stratification and terrigenous deposition well before (103 yrs) the CIE, interpreted as 